<a id='business_understanding'></a>
# 1. Business Understanding
- The purpose of this machine learning project is to predict machine wheel bearing temperatures in order to prevent machine breakdown
- There are a number of sensors which can be used as input features
- This project loosely follows the CRISP-DM methodology and is divided into the following sections: <br>
[1. Business Understanding](#business_understanding)<br>
[2. Data Understanding](#data_understanding)<br>
[3. Data Preparation](#data_preparation)<br>
[4. Modelling](#modelling)<br>
[5. Evaluation](#evaluation)<br>
[6. Deployment](#deployment)<br>

<a id='data_understanding'></a>
# 2. Data Understanding

## Steps
The notebook passes through the following steps in reaching the ultimate end-goal: building an Python model that can be used to detect anomalies in one of the sensors: `Wheel Front Temp Celsius`

[Stage 1 - Read the input files and review data](#read_input)<br>
[Stage 2 - Transform and collect statistics](#transform_collect_statistics)<br>
[Stage 3 - Visualize the data](#visualize)<br>
[Stage 4 - Find correlations in the data](#find_correlations)<br>
[Stage 5 - Train model](#train_model)<br>

## Import required packages
Even though you can import new packages anywhwere in the notebook, it is common practice move the imports to the beginning of the notebook. This to immediately show the dependencies when someone else opens it.

Additionally, when you finished working on the notebook:
* Restart the kernel (Kernel --> Restart)
* Re-run the notebook (Run --> Run All Cells)

This is to ensure the notebook will also run if cells are not executed in the exact same sequence when you developed it. Very often, the exploration and analysis of data is an iterative process and you may end up renaming variables, updating dataframe columns. By running the full notebook from start to end and validating the end result, you will avoid errors when you need to use it later.

In [ ]:
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

<a id='read_input'></a>
## Read input files
In this example, the files are directly read from the GitHub repository into a Pandas dataframe. Even though you could upload the file to the notebook server (Watson Studio or other), you will find it as easy to load the data directory from the internet.

The following 3 files have been made available:
* readings
* sensors
* devices

Once these files have been loaded into the catalog, each individual file can be imported using the Data tab to the right. 

In [ ]:
readings_df = pd.read_csv('/project_data/data_asset/readings.csv')
readings_df.head()

In [ ]:
sensors_df = pd.read_csv('/project_data/data_asset/sensor.csv')
sensors_df.head()

In [ ]:
devices_df = pd.read_csv('/project_data/data_asset/device.csv')
devices_df.head()

Now that the data has been loaded into the 3 Pandas dataframes, check that the number of records match the expectations.
* readings: ~326k entries
* sensors: 27 entries
* devices: 4 entries

In [ ]:
print('Number of readings: {}'.format(len(readings_df)))
print('Number of sensors: {}'.format(len(sensors_df)))
print('Number of devices: {}'.format(len(devices_df)))

## View the first few rows of every data set
Show the top 5 rows of every dataframe

In [ ]:
readings_df.head()

In [ ]:
sensors_df.head()

In [ ]:
devices_df.head()

<a id='data_preparation'></a>
# 3. Data Preparation

<a id='transform_collect_statistics'></a>
## Convert epoch timestamp to datetime
Extend the "readings" dataframe with a timestamp column which represents the human readable date and time for the `tsepoch` column. If you haven't noticed yet, the `tsepoch` timestamp is expressed in milliseconds.

In [ ]:
readings_df['ts']=pd.to_datetime(readings_df['tsepoch'],unit='ms')
readings_df.head()

In [ ]:
readings_df['ts'].apply(lambda x: x.day)

### Determine time span of the readings
Now that you've loaded the data, find the following properties for the readings:
* The number of seconds that the readings span
* The lowest timestamp in human readable format
* The highest timestamp in human readable format

In [ ]:
start_tsepoch=readings_df['tsepoch'].min()
end_tsepoch=readings_df['tsepoch'].max()
start_time=readings_df['ts'].min()
end_time=readings_df['ts'].max()
print('Sample data set spans readings from {} to {}'.format(start_time,end_time))
print('Number of seconds spanned by date set: {}'.format((end_tsepoch-start_tsepoch)/1000))

### Determine number of readings for each sensor
Not every sensor has the same sample rate. Later you will have to resample the readings to ensure you have a value for every time interval. Find the number of readings for every sensor and display this in a chart.

**Tip**: When using `matplotlib`, don't forget to add an instruction to tell the notebook that charts should be shown in-line.

In [ ]:
sensor_counts=readings_df.groupby('sensor_id').size()

In [ ]:
%matplotlib inline

In [ ]:
r=sensor_counts.plot(kind='bar')

### Determine statistics for each sensor and write to new "sensors" CSV file
The `sensor.csv` file contained placeholders for the count, minimum, maximum and mean of every sensor. Not all of these values were populated. You will need the sensor statistics later but you do not necessarily need to store them. If you run the notebook in the cloud, you can consider to try and persist the data in the object store, but retrieving the values in a dataframe is sufficient.

In [ ]:
sensor_statsdf=readings_df.groupby('sensor_id', as_index=False, group_keys=False)['value'].agg(['count','min','max','mean']).reset_index()
sensor_statsdf

Once you have retrieved the count, minimum, maximum and mean, join this data with the original sensors dataframe so that you can persist it as a csv file.

The resulting dataframe should have the following columns:
* sensor_id
* description
* low_value
* high_value
* mean_value

**Tip**: To be able to join, you may have to reset the index of the dataframe holding the aggregated values, or you may have to use the index of that dataframe to join with the sensors dataframe.

In [ ]:
sensors_with_stats_df=sensors_df.merge(sensor_statsdf, left_on='sensor_id', right_on='sensor_id').drop(['low_value','high_value','count'], axis=1)
sensors_with_stats_df.columns=['sensor_id','description','low_value','high_value','mean_value']
sensors_with_stats_df.head()
# We cannot write back to the file system in Watson Studio
# sensors_with_stats_df.to_csv('sensors_with_stats.csv',index=False)

<a id='visualize'></a>
## Visualization

### Determine which sensors make sense to visualize
Sensors with a constant value can be ignored and should be dropped before visualizing. Use the sensor statistics you retrieved above to determine if sensors have a constant value. Drop the readings of those sensors.

**Tip**: When dropping rows or columns, you may get a warning that the original dataframe could be affected. Use the `copy()` function to make a copy of the original dataframe before deleting rows or columns.

In [ ]:
non_zero_sensors_df=sensors_with_stats_df.loc[(sensors_with_stats_df['low_value']!=0) | \
                                           (sensors_with_stats_df['high_value']!=0)]
var_readings_df=readings_df.copy()[readings_df.sensor_id.isin(non_zero_sensors_df.sensor_id)]
var_readings_df.head()

If your code is correct, approximately 321900 readings should remain in the new dataframe.

In [ ]:
len(var_readings_df)

## Plot some of the sensors
Pick a couple of sensor IDs (for example: 14, 27 and 68) and plot the values. Use the human readable timestamp for the x-axis.

In [ ]:
plot_readings_df=var_readings_df.copy()[var_readings_df.sensor_id.isin([14,27,68])]
plot_readings_df=plot_readings_df.pivot_table(index='ts',columns='sensor_id',values='value').reset_index()
plot_readings_df.plot(x='ts')

## Down-sample the different readings
Before we can start looking at correlations between different sensors, we need to match the timestamps of the readings from the different sensos. Let's try and down-sample the variable readings to create a pivoted dataframe with a value for every sensor and every timestamp.

To lose as little detail as possible, we will down-sample to 0.2 seconds. This means there will be 864 * 5 readings for every sensor.

**Tip**: Use the pandas `resample` function and `groupby` to down-sample the readings to 200 milliseconds.

In [ ]:
readings_resample_df=var_readings_df.copy().set_index('ts').groupby('sensor_id')['value'].resample('200 ms').min().reset_index()
readings_resample_df.head()

In [ ]:
readings_resample_df.dropna(inplace=True)

Check that you more or less have the same number of samples for each sensor now.

In [ ]:
readings_resample_df.groupby('sensor_id').size()

This is what we expected. We can now pivot the table to find correlations.

<a id='find_correlations'></a>
## Find correlations between sensors
Now that we have the equivalent number of readings for every sensor, you can find correlcations between the different sensor IDs. You will need to match up the readings for different sensors with each other.

**Tip**: Use the pandas `pivot_table` function to get 1 column for every sensor. Every row will have a timestamp and value for each of the sensors.

In [ ]:
pivot_readings_df=readings_resample_df.pivot_table(index='ts',columns='sensor_id',values='value').reset_index()
pivot_readings_df.head()

The column names now reference the `sensor_id` which really is not that meaningful. Before correlating, let's give the columns some meaningful names.

**Tip**: Join the numeric column names with the `sensors` data to retrieve the names. Once you have changed the column names, show the first 5 rows of the resulting dataframe.

In [ ]:
sensor_cols=pd.DataFrame(pivot_readings_df.columns[1:])
sensor_cols=sensor_cols.astype('int64')

In [ ]:
pivot_readings_df.columns=['timestamp'] + sensor_cols.merge(sensors_with_stats_df)['description'].tolist()
pivot_readings_df.head()

### Build the correlations table

In [ ]:
readings_cov=pivot_readings_df.corr()
readings_cov

### Show correlations in heatmap
The above correlation diagram is a little difficult to read. It is better to convert this into a heatmap.

**Tip**: The Seaborn package has some nice heatmaps that are easy to use. https://seaborn.pydata.org/

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(readings_cov, annot=True, fmt='.2f',cmap='Reds')
plt.show()

From the above it is obvious that `Wheel Front Speed RPM` and `Wheel Front Temp Celsius` are strongly correlated, let's plot this in a regression plot.

In [ ]:
plt.figure(figsize=(15,10))
sns.regplot(pivot_readings_df['Wheel Front Speed RPM'], pivot_readings_df['Wheel Front Temp Celsius'],marker='+')
plt.show()

The covariance between the speed and temperature is clearly visible in the chart above.

<a id='modelling'></a>
# 4. Modelling

<a id='train_model'></a>
## Train model on the data

Now that we have found a correlation, let's try to build a model we can use for predictions.

### Import additional packages
You can choose to import additional packages here, but in the end it is recommended to move all imports to the top of the notebook.

In [ ]:
# Import pipeline libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
# Import algorithms
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet

### Split the data in a training and test set
It is best to start from the pivoted dataframe. If you didn't do so before, you first have remove any NaN values from the overall pivoted dataframe, otherwise the training or testing of the model will fail.

In [ ]:
X= pivot_readings_df.dropna().drop(['timestamp','Wheel Front Temp Celsius'], axis=1)
y = pivot_readings_df.dropna()['Wheel Front Temp Celsius']

In [ ]:
print(len(X), len(y))

Now split the dataframe into a training and test set. Please note that the independent variables (features) and dependent variables (labels) must end up in different dataframes/series.

**Tip**: SciKit Learn has a nice function that will split up a dataframe in training and testing data, and also separate features from labels.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

### Setup Model Pipelines
Use model pipelines to help select the best performing model. The pipeline dictionary below contains five different algorithms with a range of linear and ensemble methods. 

In [ ]:
pipelines = {
    'rf':make_pipeline(StandardScaler(), RandomForestRegressor()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingRegressor()),
    'ridge':make_pipeline(StandardScaler(), Ridge()),
    'lasso':make_pipeline(StandardScaler(), Lasso()),
    'enet':make_pipeline(StandardScaler(), ElasticNet())
}

### Specify Model Hyperparameter Grid

In [ ]:
mod = RandomForestRegressor()

In [ ]:
mod.get_params()

In [ ]:
hyper = {
    'rf':{'randomforestregressor__min_samples_split':[2,4,6]},
    'gb':{'gradientboostingregressor__alpha':[0.1,0.5,0.9]},
    'ridge':{'ridge__alpha':[0.1,0.5,0.9]},
    'lasso':{'lasso__alpha':[0.1,0.5,0.9]},
    'enet':{'elasticnet__alpha':[0.1,0.5,0.9]}
}

### Build and Train Pipelines

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    mod = GridSearchCV(pipeline, hyper[algo], n_jobs=-1, cv=10)
    mod.fit(X_train, y_train)
    fit_models[algo] = mod
    print('{} model has been fit'.format(algo))

<a id='evaluation'></a>
# 5. Evaluation

### Evaluate Models

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

In [ ]:
for algo, fit_model in fit_models.items():
    mod = fit_model
    yhat = mod.predict(X_test)
    print('Scores for {}, R2: {}, MAE:{}'.format(algo, r2_score(y_test,yhat), mean_absolute_error(y_test,yhat)))

### Plot Results

In [ ]:
viz = pivot_readings_df.dropna()

In [ ]:
rf = fit_models['rf'].predict(viz.drop(['timestamp', 'Wheel Front Temp Celsius'], axis=1))
gb = fit_models['gb'].predict(viz.drop(['timestamp', 'Wheel Front Temp Celsius'], axis=1))
lasso = fit_models['lasso'].predict(viz.drop(['timestamp', 'Wheel Front Temp Celsius'], axis=1))

In [ ]:
viz['rf_pred'] = rf
viz['gb_pred'] = gb
viz['lasso_pred'] = lasso

In [ ]:
viz.head()

Once you have the dataframe with these columns, plot the values. Try to improve the visualization by:
* Choosing a figure size that will fit the width of your screen
* Choose different colours for the plotted points
* Create a legend that explains the values and colours shown in the chart

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(viz['Wheel Front Speed RPM'].values,viz['Wheel Front Temp Celsius'].values,'c+')
plt.plot(viz['Wheel Front Speed RPM'].values,viz['rf_pred'].values,'y', label='Prediction Random Forest')
plt.plot(viz['Wheel Front Speed RPM'].values,viz['gb_pred'].values,'b', label='Prediction Gradient Boosted')
plt.plot(viz['Wheel Front Speed RPM'].values,viz['lasso_pred'].values,'g', label='Prediction Lasso')
plt.legend(loc='upper left')
plt.xlabel('Wheel Front Speed RPM')
plt.ylabel('Wheel Front Temperature Celsius')
plt.show()

<a id='deployment'></a>
# 6. Deployment
We can now deploy the Gradient Boosted model as the highest performing model. 

In [ ]:
from ibm_watson_machine_learning import APIClient
import json
import numpy as np 
import os 

In [ ]:
url = 'YOUR ENV URL HERE'
token = os.environ['USER_ACCESS_TOKEN']

In [ ]:
wml_credentials = {
    "token": token, 
    "url": url,
    "instance_id": 'openshift',
    "version": '3.5'
}

In [ ]:
wml_client = APIClient(wml_credentials)
wml_client.spaces.list()

In [ ]:
SPACE_ID="YOUR SPACE ID"
wml_client.set.default_space(SPACE_ID)

In [ ]:
MODEL_NAME = "IOT Forecast"
DEPLOYMENT_NAME = "IOT Forecast"
BEST_MODEL = fit_models['gb']

In [ ]:
# Set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('default_py3.7')

# Setup model meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

#Save model
model_details = wml_client.repository.store_model(
    model=BEST_MODEL, 
    meta_props=model_props, 
    training_data=X_train.head(), 
    training_target=y_train.head()
)

In [ ]:
model_details

In [ ]:
model_uid = wml_client.repository.get_model_uid(model_details); model_uid

In [ ]:
# Set meta
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# Deploy
deployment = wml_client.deployments.create(
    artifact_uid=model_uid, 
    meta_props=deployment_props 
)

# Output result
deployment

In [ ]:
wml_client.deployments.list()

## Scoring

In [ ]:
deployment

In [ ]:
deployment_uid = wml_client.deployments.get_uid(deployment)
payload = {"input_data":
           [
               {"fields":X_train.columns.to_numpy().tolist(), "values":X_train.to_numpy().tolist()}
           ]
          }

In [ ]:
result = wml_client.deployments.score(deployment_uid, payload); result